# 함수 데커레이터와 클로저 
* 함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해줍니다. 강력한 기능이지만, 데커레이더를 자유롭게 사용하기 위해서는 클로저 것을 먼저 알아야 합니다. 
* nonlocal 키워드 

## 살펴볼 내용
* 파이썬이 데커레이터 구문을 평가하는 방식 
* 변수가 지역 변수인지 파있너이 판단하는 방식 
* 클로저의 존재 이유와 작동 방식 
* nonlocal로 해결할 수 있는 문제 

## 주제 
* 잘 작동하는 데커레이터 구현하기 
* 표준 라이브러리에서 제공하는 재미있는 데커레이터들 
* 매개변수화된 데커레이터 구현하기 

## 데커레이터 기본 지식 
* 데커레이터는 다른 함수를 인수로 받는 callable이다. 
* 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 callable 객체로 대체한다. 

* 아래 코드에서처럼 decorate라는 이름의 데커레이터가 있다고 가정해보자 

In [1]:
import decorator

In [3]:
def deco(func):
    def inner(): #함수안에 함수 선언가능 객체이므로
        print ("running inner()")
    return inner

* deco()가 inner() 함수 객체를 반환한다. 

In [4]:
@deco
def target():
    print ('running target()')

* target()을 deco로 데커레이트했다

In [5]:
target()
#기능을 추가하고싶을때 @deco 를 적고 함수를 입력한다
# 모듈추가하고싶을때 기존함수에서

running inner()


* target의 값을 조사해보면 inner()를 가리키고 있음을 알 수 있음

In [5]:
target 

<function __main__.deco.<locals>.inner()>

* 데커레이터는는 다른 함수를 인수로 전달해서 호출하는 일반적인 콜러블과 동일함 
* 그렇지만 런타임에 프로그램 행위를 변경하는 메타프로그래밍을 할 때 데커레이터가 상당히 편리함 
* 1. 데커레이터는 데커레이트된 함수를 다른 함수로 대체하는 능력이 있음 
* 2. 데커레이터는 모듈이 로딩될 때 바로 실행 

## 데커레이터를 실행하는 시점

In [6]:
%run registration.py

running register(<function f1 at 0x00000222D68E3B80>)
running register(<function f2 at 0x00000222D68E3DC0>)
running main()
registry -> [<function f1 at 0x00000222D68E3B80>, <function f2 at 0x00000222D68E3DC0>]
running f1()
running f2()
running f3()


* register()는 모듈 내의 다른 어떠한 함수보다 먼저 실행(두번) 된다. 
* register()가 호출될 때 데커레이된 함수를 인수로 받는다. 
* 모듈이 로딩된 후 registry는 데커레이터된 두 개의 함수 f1()와 f2()에 대한 참조를 가진다.

In [7]:
import registration

running register(<function f1 at 0x00000222D68E3D30>)
running register(<function f2 at 0x00000222D68F3430>)


In [8]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

* 함수 데커레이터는 모듈이 임포트되자마자 실행되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있음 

## 데커레이터로 개선한 전략 패턴 
#### 디자인 패턴 
* 일련의 알고리즘을 정의하고 각각을 하나의 클래스 안에 넣어서 교체하기 쉽게 만든다. 전략을 이용하면 사용하는 클라이언트에 따라 알고리즘을 독립적으로 변경할 수 있다. 

### namedtuple
* 튜플의 특징은 한번 만들어진 데이터는 변경되지 않은 특징을 가지고 있습니다. 이런 특징을 바탕으로 인덱스가 아닌 키값을 사용해서 데이터를 접근할 수 있습니다. 
* 아래 예제를 보시면 Customer라는 튜플의 이름으로 name과 fidelity라는 두 키값을 설정하여 값을 매핑시키는 예제입니다. 

In [79]:
from collections import namedtuple

In [80]:
Customer = namedtuple("Customer", "name fidelity")

In [81]:
joe = Customer("John Doe", 0)

In [82]:
joe

Customer(name='John Doe', fidelity=0)

### 예제 

In [6]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

In [7]:
@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 100 else 0

In [8]:
help(fidelity)

Help on function fidelity in module __main__:

fidelity(order)
    충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용



In [30]:
@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

In [31]:
@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [32]:
def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

## 변수 범위 규칙 

In [9]:
def f1(a):
    print (a)
    print (b)

* b라는 변수가 정의되지 않아서 오류가 발생 

In [10]:
f1(3)

3


NameError: name 'b' is not defined

* 아래 코드에서 b를 6으로 할당. 해당 b라는 변수는 전역변수로 설정되어 있다. 

In [11]:
b = 6

In [12]:
f1(3)

3
6


* 함수 본체 안에서 할당된 변수는 지역 변수로 판단하고 해당 변수가 없을 경우 전역에서 변수를 찾는다. 

-------

In [13]:
b = 6
def f2(a):
    print (a)
    print (b)
    b = 9

In [14]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

* 위의 코드에서 에러나는 이유는 파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역 변수로 판단한다. 

----
* 아래 예제는 global 키워드를 사용해 b 변수를 전역으로 변경한 예제이다. 

In [15]:
b = 6 
def f3(a):
    global b 
    print (a)
    print (b)
    b = 9

In [16]:
f3(3)

3
6


In [17]:
b

9

In [18]:
f3(3)

3
9


In [19]:
b = 30

In [20]:
b

30

In [21]:
from dis import dis 

In [22]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [23]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


* 
LOAD_GLOBAL(namei)

    co_names[namei]라는 이름의 전역을 스택에 로드합니다.
    
* 
LOAD_FAST(var_num)

    지역 co_varnames[var_num]에 대한 참조를 스택으로 푸시합니다.

